In [3]:
# pip install SpeechRecognition
# pip install pyttsx3
# pip install sounddevice
# pip install playsound
# pip install pydub
# pip install pydub gtts
# pip install transformers



In [1]:
import numpy as np
import time
import os
import speech_recognition as sr
from gtts import gTTS
import pyttsx3
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [2]:
    # Checkpoint
    checkpoint = "microsoft/DialoGPT-medium"

    # Download and cache tokenizer
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    # Download and cache pre-trained model
    model = AutoModelForCausalLM.from_pretrained(checkpoint)


In [3]:
class ChatBot():
    # initialize
    def __init__(self):
        self.chat_history_ids = None
        self.bot_input_ids = None
        self.end_chat = False
        self.welcome()

    def welcome(self):
        print("Initializing ChatBot ...")
        time.sleep(2)
        print('Say "bye" or "quit" or "exit" to end chat\n')
        time.sleep(3)

        greeting = np.random.choice([
            "Welcome, I am ChatBot, here for your kind service",
            "Hey, Great day! I am your virtual assistant",
            "Hello, it's my pleasure meeting you",
            "Hi, I am a ChatBot. Let's chat!"
        ])
        print("ChatBot >>  " + greeting)

    def get_user_input(self):
        r = sr.Recognizer()
        mic = sr.Microphone()

        with mic as source:
            print("Listening...")
            audio = r.listen(source)

        try:
            text = r.recognize_google(audio)
            return text
        except sr.UnknownValueError:
            print("Sorry, I could not understand you.")
            return ""
        except sr.RequestError:
            print("Sorry, I'm having trouble processing your request. Please try again.")
            return ""

    def speak(self, text):
        engine = pyttsx3.init()
        engine.say(text)
        engine.runAndWait()

    def bot_response(self):
        if self.chat_history_ids is not None:
            self.bot_input_ids = torch.cat([self.chat_history_ids, self.new_user_input_ids], dim=-1)
        else:
            self.bot_input_ids = self.new_user_input_ids

        self.chat_history_ids = model.generate(self.bot_input_ids, max_length=1000, \
                                               pad_token_id=tokenizer.eos_token_id)

        response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[-1]:][0], \
                                    skip_special_tokens=True)

        if response == "":
            response = self.random_response()

        print('ChatBot >>  ' + response)
        self.speak(response)

    def random_response(self):
        i = -1
        response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                                    skip_special_tokens=True)

        while response == '':
            i = i - 1
            response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                                        skip_special_tokens=True)

        if response.strip() == '?':
            reply = np.random.choice(["I don't know",
                                      "I am not sure"])

        else:
            reply = np.random.choice(["Great",
                                      "Fine. What's up?",
                                      "Okay"
                                      ])
        return reply

In [4]:
bot = ChatBot()

while True:
    user_input = bot.get_user_input()

    if user_input.lower().strip() in ['bye', 'quit', 'exit']:
        bot.end_chat = True
        bot.new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
        print('User: ' + user_input)  # Print the user's question
        bot.bot_response()
        print('ChatBot >> See you soon! Bye!')
        time.sleep(1)
        print('\nQuitting ChatBot ...')
        break
    elif user_input != "":
        bot.new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
        print('User: ' + user_input)  # Print the user's question
        bot.bot_response()

Initializing ChatBot ...
Say "bye" or "quit" or "exit" to end chat

ChatBot >>  Hello, it's my pleasure meeting you
Listening...
Sorry, I could not understand you.
Listening...
Sorry, I could not understand you.
Listening...
Sorry, I could not understand you.
Listening...
Sorry, I could not understand you.
Listening...
Sorry, I could not understand you.
Listening...
Sorry, I could not understand you.
Listening...
User: hi how are you
ChatBot >>  I'm good, how are you?
Listening...
Sorry, I could not understand you.
Listening...
Sorry, I could not understand you.
Listening...
Sorry, I could not understand you.
Listening...
Sorry, I could not understand you.
Listening...
Sorry, I could not understand you.
Listening...


KeyboardInterrupt: 